## Load the tasks Data

In [23]:
import pandas as pd
task_df =  pd.read_csv('C:/Users/Arwa/Desktop/datasets/Ex2/Jobs_dataset_tokenized_Ex2.csv')

In [24]:
task1 = task_df.loc[52]
task2 = task_df.loc[53]

print(task1.loc['Job_Title'])
print(task2.loc['Job_Title'])

Wordpress Expert install theme
install wordpress theme plugins two


## compute the simiality between titles using the cosine similarity 

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

if task1.loc['Job_Title']!= '' and task2.loc['Job_Title']!= '':
    data = [
        task1.loc['Job_Title'],
        task2.loc['Job_Title']
            ]
    print(data)

    # Vectorise the data
    vec = TfidfVectorizer()

    X = vec.fit_transform(data)  # `X` will now be a TF-IDF representation of the data, the first row of `X` corresponds to the first sentence in `data`

    # Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
    S = cosine_similarity(X)
    Title_Sim = S[1,0]
else:
    Title_Sim = 0
print('similarity of titles:', Title_Sim)

['Wordpress Expert install theme', 'install wordpress theme plugins two']
similarity of titles: 0.5101490193104813


## the concept similairty method -- Taxonomy based simialrity 

In [27]:

from ete3 import TreeNode,Tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def concept_similarity_measure_ex2(C1, C2):
    
    taxonomy = Tree("skills_small_taxonomy_tree_level_score.nw")
    # print("C1",C1,"\n","C2",C2)
    

    N1 = 0 # the distance from Concept 1 to the least common subsumer
    N2 = 0 # the distance from Concept 2 to the least common subsumer
    N = 0  # the distance from  the least common subsumer to the root
    """ -----------------L   he shortest path between the tow concepts------------------------------"""


    node1 = taxonomy.search_nodes(name=C1)
    node2 = taxonomy.search_nodes(name=C2)
    # print('node1', node1)
    # print('node2', node2)
    # if the skill is not found in the taxonomy
    if node1 == [] or node2 == []:
        # print('skills not in taxonomy')
        # print(C1, C2)
        data = [C1, C2]

        # Vectorise the data
        vec = TfidfVectorizer()

        X = vec.fit_transform(
            data)  # `X` will now be a TF-IDF representation of the data, the first row of `X` corresponds to the first sentence in `data`

        # Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
        S = cosine_similarity(X)
        similarity = S[0, 1]


        # print('simmmms',similarity)
        l1 = l2 = 1.0  # how much should it be

    else:
        node1 = node1[0]
        node2 = node2[0]
        common = node1.get_common_ancestor(node2)
        # print(common.is_root())
        # print("common is ",common.name)

        """ ------------------N the distance from root node to the least common subsumer-------------------"""
        root = taxonomy.get_tree_root()
        N = taxonomy.get_distance(common, root, topology_only=False)
        # print("N = the distance between the common ancestor", common.name, "AND  ROOT IS ", N)

        """ ----------------N1 the distance from Concept 1 to the least common subsumer--------------------"""
        N1 = taxonomy.get_distance(C1, common, topology_only=False)
        # print("N1 = the distance between",C1,"AND  ROOT IS ",N1)

        """ ----------------N1 the distance from Concept 2 to the least common subsumer--------------------"""
        N2 = taxonomy.get_distance(C2, common, topology_only=False)
        # print("N2 = the distance between",C2, "AND  ROOT IS ", N2)

        """ -------------------------------COMPUTE THE MEASURE FORMULA----------------------------------------"""

        similarity = (2*N)/(N1+N2+(2*N))

        
        l1 = node1.level_score
        l2 = node2.level_score
        similarity = round(similarity, 2)
        print("similarity between ", C1,"and",C2, "is", similarity)
        print("level score  ",C1,'  is', l1)
        print("level score  ",C2,'  is', l2)
        print("---------------------------------------------------")
    return similarity,l1,l2 

## set of skills similarity between the 2 tasks 

In [28]:
import numpy as np
def is_nan(x):
    return (x is np.nan)

In [29]:
skills_t1 = task1.loc['Required_Skills 1':'Required_Skills 5']
skills_t2 = task2.loc['Required_Skills 1':'Required_Skills 5']


# skills_t1.sort()
# skills_t2.sort()
print(skills_t1)
print(skills_t2)

Required_Skills 1             Plgin
Required_Skills 2    Website Design
Required_Skills 3         WordPress
Required_Skills 4               NaN
Required_Skills 5               NaN
Name: 52, dtype: object
Required_Skills 1        Plgin
Required_Skills 2    WordPress
Required_Skills 3          NaN
Required_Skills 4          NaN
Required_Skills 5          NaN
Name: 53, dtype: object


## compute the similairty among the 2 sets of tasks using the concept similaity method--taxonomy based

In [30]:

i = 0
import pandas as pd  

sim_df = pd.DataFrame()

for s1 in skills_t1:
    
    sim_list = []
    if not is_nan(s1):
        
        for s2 in skills_t2:
            
            if not is_nan(s2):
                print("s1", s1)
                print("s2",s2)
                sim ,l1, l2 = concept_similarity_measure_ex2(s1, s2) # this function returns a tuple (sim, l1,l2)
                sim = float(sim)  # convert to floats
                sim_df.loc[s1,s2] = sim
            
                sim_list.append(sim)
        print('sim_list of round #',i, "is  ", sim_list)
        print("____________________________________________________________________________________________")
        i = i + 1 
       
       

s1 Plgin
s2 Plgin
similarity between  Plgin and Plgin is 1.0
level score   Plgin   is 1.0
level score   Plgin   is 1.0
---------------------------------------------------
s1 Plgin
s2 WordPress
similarity between  Plgin and WordPress is 0.91
level score   Plgin   is 1.0
level score   WordPress   is 0.8
---------------------------------------------------
sim_list of round # 0 is   [1.0, 0.91]
____________________________________________________________________________________________
s1 Website Design
s2 Plgin
similarity between  Website Design and Plgin is 0.67
level score   Website Design   is 0.5
level score   Plgin   is 1.0
---------------------------------------------------
s1 Website Design
s2 WordPress
similarity between  Website Design and WordPress is 0.75
level score   Website Design   is 0.5
level score   WordPress   is 0.8
---------------------------------------------------
sim_list of round # 1 is   [0.67, 0.75]
_______________________________________________________________

## we have the similaity matix M 

In [31]:
sim_df

,Plgin,WordPress
Plgin,1.00,0.91
Website Design,0.67,0.75
WordPress,0.91,1.00


## construct the similarity vectors 

In [36]:
import pandas as pd

sim_vec_df = pd.DataFrame()

for s in skills_t1:
    if not is_nan(s):
        sim_vec_df.loc['t1', s] = 0.0
for s in skills_t2:
    if not is_nan(s):
        sim_vec_df.loc['t1', s] = 0.0
for s in skills_t1:
    if not is_nan(s):
        sim_vec_df.loc['t2', s] = 0.0
for s in skills_t2:
    if not is_nan(s):
        sim_vec_df.loc['t2', s] = 0.0


sim_vec_df

,Plgin,Website Design,WordPress
t1,0.0,0.0,0.0
t2,0.0,0.0,0.0


In [37]:
# set the skills of t1 to 1 in v1
for skill in skills_t1:
    if not is_nan(skill):
        sim_vec_df.loc['t1', skill] = 1.0

# set the skills of t2 to 1 in v2
for skill in skills_t2:
    if not is_nan(skill):
        sim_vec_df.loc['t2', skill] = 1.0

    # set the skills of t2 to the max in v1
for skill in skills_t2:
    if not is_nan(skill):
        sim_vec_df.loc['t1', skill] = sim_df[skill].max()

# set the skills of t1 to t2 in v2

for skill in skills_t1:
    if not is_nan(skill):
        sim_vec_df.loc['t2', skill] = sim_df.loc[skill].max()

sim_vec_df

,Plgin,Website Design,WordPress
t1,1.0,1.00,1.0
t2,1.0,0.75,1.0


In [40]:
from sklearn.metrics.pairwise import cosine_similarity
v1 = sim_vec_df.loc['t1'].values
v1


array([1., 1., 1.])

In [41]:
v1= v1.reshape(1, -1)
v1

array([[1., 1., 1.]])

In [39]:
v2 = sim_vec_df.loc['t2'].as_matrix()
v2=v2.reshape(1, -1)
v2

array([[1.  , 0.75, 1.  ]])

In [14]:
skill_similarity = cosine_similarity(v1,v2)
skill_similarity

array([[0.94839793]])

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


data =['CSS HTML PHP WordPress Plgin','Plgin Website Design WordPress SQL'] 
  
   
# Vectorise the data
vec = TfidfVectorizer()

"""`X` will now be a TF-IDF representation of the data, the first row 
of `X` corresponds to the first sentence in `data`""" 

X = vec.fit_transform(data)  

print("X", X)
# Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
S = cosine_similarity(X)
Skills_Sim = S[1,0]

print('similarity of skills:', Skills_Sim)

X   (0, 0)	0.4992213265230509
  (0, 2)	0.4992213265230509
  (0, 3)	0.4992213265230509
  (0, 7)	0.35520008546852583
  (0, 4)	0.35520008546852583
  (1, 7)	0.35520008546852583
  (1, 4)	0.35520008546852583
  (1, 6)	0.4992213265230509
  (1, 1)	0.4992213265230509
  (1, 5)	0.4992213265230509
similarity of skills: 0.25233420143369617



## Matrix factorozation Using SVD 

In [ ]:
from scipy.linalg import svd


U, S, V = svd(sim_df, full_matrices=False)
U.shape, S.shape, V.shape

In [32]:
# reconstructing the original matrix 
np.dot(U.dot(np.diag(S)), V)

array([[0.55, 0.75, 0.6 , 0.25],
       [0.6 , 0.86, 0.67, 0.29],
       [0.8 , 0.86, 0.89, 0.29],
       [0.91, 0.75, 1.  , 0.25],
       [1.  , 0.67, 0.91, 0.22]])

In [33]:
print("U contain information on how your words are related to each other ")
print(U)
print("\n\n")
print("S  contains information on how much information from the underlying table is contained in each column")
print(S)
print("\n\n")
print("V  ")
print(V)

U contain information on how your words are related to each other 
[[-0.35826264 -0.44665345  0.2099729  -0.71208919]
 [-0.40211295 -0.57011472  0.17368125  0.30636737]
 [-0.4794394  -0.13167309 -0.28364323  0.54110332]
 [-0.49820434  0.34548079 -0.64831706 -0.31933139]
 [-0.48154268  0.58204393  0.65190315  0.06559481]]



S  contains information on how much information from the underlying table is contained in each column
[3.13049790e+00 3.83341068e-01 8.61764853e-02 2.47361447e-03]



V  
[[-0.56117986 -0.55042965 -0.59015612 -0.18390248]
 [ 0.53050603 -0.7550654   0.28168854 -0.26285199]
 [ 0.63490512  0.15607151 -0.75632563  0.02255465]
 [ 0.02315199 -0.3202267  -0.01840796  0.94687908]]


In [86]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components=3)
svd.fit(sim_df)  

print("The variance of the training samples transformed by a projection to each component")
print(svd.explained_variance_,"\n")  
print("Percentage of variance explained by each of the selected components.")
print(svd.explained_variance_ratio_,"\n") 
print(svd.explained_variance_ratio_.sum(),"\n")  
print("The singular values corresponding to each of the selected components.")
print(svd.singular_values_)  

The variance of the training samples transformed by a projection to each component
[0.02882949 0.0291032  0.00148209] 

Percentage of variance explained by each of the selected components.
[0.4852142  0.48982098 0.02494428] 

0.999979461335971 

The singular values corresponding to each of the selected components.
[3.1304979  0.38334107 0.08617649]


In [81]:
S[0] #highst singular value 

3.1304979029220643

In [66]:
import pandas as pd
import numpy as np 
from sklearn import linear_model
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import normalize
from scipy.linalg import svd

In [67]:
# List of explained variances
tsvd_var_ratios = svd.explained_variance_ratio_

AttributeError: 'function' object has no attribute 'explained_variance_ratio_'

In [175]:
S

array([0.4852142 , 0.48982098])

In [72]:
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [73]:
# Run function
no_components = select_n_components(tsvd_var_ratios, 0.99)
no_components

NameError: name 'tsvd_var_ratios' is not defined

In [74]:
svd = TruncatedSVD(n_components = no_components)
x = svd.fit(sim_df).transform(sim_df)
x

NameError: name 'no_components' is not defined

In [75]:
U = svd.fit_transform(sim_df)
S = svd.explained_variance_ratio_
VT = svd.components_

AttributeError: 'function' object has no attribute 'fit_transform'

In [76]:
U

array([[-0.35826264, -0.44665345,  0.2099729 , -0.71208919],
       [-0.40211295, -0.57011472,  0.17368125,  0.30636737],
       [-0.4794394 , -0.13167309, -0.28364323,  0.54110332],
       [-0.49820434,  0.34548079, -0.64831706, -0.31933139],
       [-0.48154268,  0.58204393,  0.65190315,  0.06559481]])

In [176]:
VT

array([[ 0.56117986,  0.55042965,  0.59015612,  0.18390248],
       [ 0.53050603, -0.7550654 ,  0.28168854, -0.26285199]])

In [177]:
# reconstructing the reduced matrix 
np.dot(U.dot(np.diag(S)), VT)

array([[0.2608948 , 0.36286226, 0.297531  , 0.12212213],
       [0.28597501, 0.41702873, 0.33030935, 0.14046478],
       [0.39556311, 0.41951881, 0.42281686, 0.14042579],
       [0.45908895, 0.36755815, 0.46487595, 0.12211752],
       [0.46845097, 0.32008831, 0.46245237, 0.10578759]])

In [178]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components=no_components)
svd.fit(X)  


print(svd.explained_variance_ratio_)  

print(svd.explained_variance_ratio_.sum())  

print(svd.singular_values_) 

[8.05204987e-31 1.00000000e+00]
1.0000000000000002
[1.22888121 0.69989355]


In [179]:
print(svd.explained_variance_ratio_.sum()/no_components ) 

0.5000000000000001


In [125]:
def test_algorithms():
    svd_a = TruncatedSVD(30, algorithm="arpack")
    svd_r = TruncatedSVD(30, algorithm="randomized", random_state=42)
 
    Xa = svd_a.fit_transform(X)[:, :6]
    Xr = svd_r.fit_transform(X)[:, :6]
    assert_array_almost_equal(Xa, Xr, decimal=5)
 
    comp_a = np.abs(svd_a.components_)
    comp_r = np.abs(svd_r.components_)
    # All elements are equal, but some elements are more equal than others.
    assert_array_almost_equal(comp_a[:9], comp_r[:9])
    assert_array_almost_equal(comp_a[9:], comp_r[9:], decimal=2)

In [126]:
test_algorithms()

ValueError: k must be between 1 and min(A.shape), k=30

## then the total task similairty score can be found by 

In [51]:
C1 = 0.3
C2 = 0.7
task_2_task_similarity_score = C1*Title_Sim + C2*skills_sim
task_2_task_similarity_score = round(task_2_task_similarity_score, 1)
print(task_2_task_similarity_score)

0.4


## 2nd Algorithm 

In [116]:
weight_list = []
sim_max = []
ave_list = [] 
i = 0
import pandas as pd  
scaler = 0 
total_c = 0 
sim_df = pd.DataFrame()
ave = 0 
for s1 in skills_t1:
    i = i+1
    sim_list = []
    if not is_nan(s1):
        
        for s2 in skills_t2:
            
            if not is_nan(s2):
                print("s1", s1)
                print("s2",s2)
                sim ,l1, l2 = concept_similarity_measure_ex2(s1, s2) # this function returns a tuple (sim, l1,l2)
                sim = float(sim)  # convert to floats
                l1 = float(l2)
                l2 = float(l1)
                sim_df.loc[s1,s2] = sim
            
                sim_list.append(sim)
        print('sim_list of round #',i, "is  ", sim_list)
        sim_list = np.array(sim_list)
        
#         maxima = sim_list.argmax()
#         print("argmax = ", maxima)
                
        for k in sim_list:
            if k >= 0.8025:
                total_c= total_c + 1 
                
        print("____________________________________________________________________")

        scaler = scaler + len(sim_list) 
        
print("total count of values > 0.8 = ", total_c)
skills_sim = round(total_c /scaler,1) 
print("length of all the lists ", scaler)
print("SKILLS SIM total count / total number of cells = ", skills_sim)
print("---------------------------------------------------")


s1 CSS
s2 Plgin
similarity between  CSS and Plgin is 0.55
level score   CSS   is 0.8
level score   Plgin   is 1.0
---------------------------------------------------
s1 CSS
s2 Website Design
similarity between  CSS and Website Design is 0.75
level score   CSS   is 0.8
level score   Website Design   is 0.5
---------------------------------------------------
s1 CSS
s2 WordPress
similarity between  CSS and WordPress is 0.6
level score   CSS   is 0.8
level score   WordPress   is 0.8
---------------------------------------------------
s1 CSS
s2 SQL
similarity between  CSS and SQL is 0.25
level score   CSS   is 0.8
level score   SQL   is 0.5
---------------------------------------------------
sim_list of round # 1 is   [0.55, 0.75, 0.6, 0.25]
____________________________________________________________________
s1 HTML
s2 Plgin
similarity between  HTML and Plgin is 0.6
level score   HTML   is 0.7
level score   Plgin   is 1.0
---------------------------------------------------
s1 HTML
s2 Websi

## 3rd Algorithm 

In [119]:

weight_list = []
sim_max = []
ave_list = [] 
i = 0
import pandas as pd  
scaler = 0 
total_valid_values = 0 
sim_df = pd.DataFrame()
ave = 0 
for s1 in skills_t1:
    c = 0 
    j = 0 
    i = i+1
    sim_list = []
    if not is_nan(s1):
        for s2 in skills_t2:
            
            if not is_nan(s2):
                print("s1", s1)
                print("s2",s2)
                sim ,l1, l2 = concept_similarity_measure_ex2(s1, s2) # this function returns a tuple (sim, l1,l2)
                sim = float(sim)  # convert to floats

                sim_df.loc[s1,s2] = sim
            
                sim_list.append(sim)
        print('sim_list of round #',i, "is  ", sim_list)
        print("________________________________________________________________")
        sim_list = np.array(sim_list)
        

                
        for k in sim_list:
            if k >= 0.8025:
                total_valid_values = total_valid_values + k 

        scaler = scaler + len(sim_list) 

print("total sum of values > 0.8 = ", total_valid_values)
skills_sim = round(total_valid_values /scaler,1) 
print("length of all the lists ", scaler)
print("SKILLS SIM total count /length of list = ", skills_sim)
print("---------------------------------------------------")


s1 CSS
s2 Plgin
similarity between  CSS and Plgin is 0.55
level score   CSS   is 0.8
level score   Plgin   is 1.0
---------------------------------------------------
s1 CSS
s2 Website Design
similarity between  CSS and Website Design is 0.75
level score   CSS   is 0.8
level score   Website Design   is 0.5
---------------------------------------------------
s1 CSS
s2 WordPress
similarity between  CSS and WordPress is 0.6
level score   CSS   is 0.8
level score   WordPress   is 0.8
---------------------------------------------------
s1 CSS
s2 SQL
similarity between  CSS and SQL is 0.25
level score   CSS   is 0.8
level score   SQL   is 0.5
---------------------------------------------------
sim_list of round # 1 is   [0.55, 0.75, 0.6, 0.25]
________________________________________________________________
s1 HTML
s2 Plgin
similarity between  HTML and Plgin is 0.6
level score   HTML   is 0.7
level score   Plgin   is 1.0
---------------------------------------------------
s1 HTML
s2 Website D